In [24]:
import nb_setup
import importlib

importlib.reload(nb_setup)
nb_setup.init()

Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [29]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)
def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            print(coin)
            result = info.candles_snapshot(coin, '15m', start, end)
            _candles = candles.CandleList.from_response(result)
            _candles.to_csv()
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(["BTC"])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 13.1 µs
BTC


In [25]:
import pandas as pd
import models.backtesting.backtesting as backtesting
import models.backtesting.BollingerBandsReEntryStrategy as bollingerBandsReEntryStrategy

importlib.reload(bollingerBandsReEntryStrategy)
importlib.reload(backtesting)
BackTesting = backtesting.BackTesting
EXPECTED_DTYPES = backtesting.EXPECTED_DTYPES


df = pd.read_csv("data/BTC.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.astype(EXPECTED_DTYPES)

df.shape
df.describe()

,datetime,number_of_trades,close,high,low,open,volume,tic,toc
count,2881,2881.000000,2881.000000,2881.000000,2881.000000,2881.000000,2881.000000,2.881000e+03,2.881000e+03
mean,2024-04-10 00:00:00,1139.971191,67367.234988,67502.716071,67222.094759,67369.387713,70.065610,1.712707e+12,1.712708e+12
min,2024-03-26 00:00:00,186.000000,59663.000000,60336.000000,59571.000000,59650.000000,4.390790,1.711411e+12,1.711412e+12
25%,2024-04-02 12:00:00,620.000000,64980.000000,65131.000000,64866.000000,64980.000000,27.537690,1.712059e+12,1.712060e+12
50%,2024-04-10 00:00:00,903.000000,67585.000000,67746.000000,67466.000000,67590.000000,47.012040,1.712707e+12,1.712708e+12
75%,2024-04-17 12:00:00,1413.000000,69969.000000,70063.000000,69859.000000,69968.000000,87.033700,1.713355e+12,1.713356e+12
max,2024-04-25 00:00:00,10788.000000,72764.000000,72836.000000,72444.000000,72765.000000,1107.488910,1.714003e+12,1.714004e+12
std,NaN,790.662951,2842.644966,2815.767203,2873.941369,2842.681755,71.535871,7.486356e+08,7.486356e+08


In [26]:
# Create a strategy
bt = BackTesting()
INITAL_CAPITAL = 200
BUY_EQUITY = 10
LEVERAGE = 20
STOP_LOSS_PERCENTAGE = 0.8 # 0 - 1
K = 2
bt.run_backtesting(
    df,
    strategy=bollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy(
        initial_capital=INITAL_CAPITAL,
        buy_equity=BUY_EQUITY,
        leverage=LEVERAGE,
        k=K,
        stop_loss_percentage=STOP_LOSS_PERCENTAGE
    ),
    # play_mode=True,
)

Validating data...
Data validated successfully.
Start Running backtesting...
Data shape: (2881, 13)
Strategy: <models.backtesting.BollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy object at 0x125dee850>
Processing row 0...
Portfolio Value:  (1, 6) : 200
Processing row 1...
shape(self.data):  (2, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (2, 6) : 200
Processing row 2...
shape(self.data):  (3, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (3, 6) : 200
Processing row 3...
shape(self.data):  (4, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (4, 6) : 200
Processing row 4...
shape(self.data):  (5, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (5, 6) : 200
Processing row 5...
shape(self.data):  (6, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (6, 6) : 200
Processing row 6...
shape(self.data):  (7, 13) states:  nan nan
No valid trade condition met.
Portfolio Value: 

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:155: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/portfolio.py:249: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



shape(self.data):  (23, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (23, 6) : 200
Processing row 23...
shape(self.data):  (24, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (24, 6) : 200
Processing row 24...
shape(self.data):  (25, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (25, 6) : 200
Processing row 25...
shape(self.data):  (26, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (26, 6) : 200
Processing row 26...
shape(self.data):  (27, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (27, 6) : 200
Processing row 27...
shape(self.data):  (28, 13) states:  BollingerBandsState.WITHIN_BANDS 

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:282: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



shape(self.data):  (48, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (48, 6) : 201.2386944553676759732599292
Processing row 48...
shape(self.data):  (49, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (49, 6) : 201.0813999213527329925285096
Processing row 49...
shape(self.data):  (50, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (50, 6) : 200.8960170776922644795236223
Processing row 50...
shape(self.data):  (51, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (51, 6) : 200.4578394472220661760575249
Processing row 51...
shape(self.data):  (52, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (

In [27]:
bt.strategy.portfolio.holdings

{'BTC': [Symbol: BTC, Entry Price: 63923, Equity: 10, Side: Side.LONG, Leverage: 20, Size: 0.003128764294541870688171706584]}

In [28]:
# display(bt.strategy.history)
# display(bt.strategy.portfolio.portfolio_history)
bt.strategy.portfolio.info()

Start Time:  2024-03-26 00:00:00
End Time:  2024-04-25 00:00:00
Cash:  252.1943886000088333070713324
Holdings: 
{'BTC': [Symbol: BTC, Entry Price: 63923, Equity: 10, Side: Side.LONG, Leverage: 20, Size: 0.003128764294541870688171706584]}
Evaluation:  263.5929462396690495046840852
PNL:  63.5929462396690495046840852
PNL Percentage:  31.79647311983452475234204260
Max DrawDown Percentage:  -17.50777602563175668200320624
Max Return Percentage:  32.00774951899937154900057935


In [29]:
from settings import DATA_DIR
name = f"{INITAL_CAPITAL}-{BUY_EQUITY}-{LEVERAGE}x-{K}std-{STOP_LOSS_PERCENTAGE}SL_BTC_BollingerBandsReEntryStrategy"
bt.strategy.history.to_csv(DATA_DIR / f"./{name}-history.csv")
bt.strategy.portfolio.portfolio_history.to_csv(
    DATA_DIR / f"./{name}-portfolio-history.csv"
)

In [30]:
bt.strategy.plot_history()

In [31]:
bt.strategy.plot_performance()